# Imports

In [1]:
from tensorflow.keras import regularizers

from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation

import matplotlib.pyplot as plt

%matplotlib inline

import numpy as np
from sklearn.model_selection import train_test_split


#!pip install tensorflow-hub
#!pip install tensorflow-datasets
import tensorflow_hub as hub

import tensorflow as tf

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

import pandas as pd
import spacy
# !pip install spacy
# !python -m spacy download en_core_web_sm
# import spacy
# nlp = spacy.load('en_core_web_sm')

Version:  2.2.0
Eager mode:  True
Hub version:  0.10.0
GPU is available


In [10]:
#read in dataset
df = pd.read_csv('final_aggregated_data_set_Oct_26.csv')

In [12]:
df

,Unnamed: 0,date,depression,anxiety,government,politics,democracy,title_text,avg_tone,pos_words,neg_words,polarity,act_ref_density,self_group_density,word_count,depression_binary,shifted_12h
0,0,2015-03-02 10:00:00,15.0,16.0,71.0,4.0,2.0,america clean energy laggardanswer resound myr...,0.350631,2.734923,2.384292,5.119215,16.760168,0.420757,1186.000000,0,0.0
1,5,2015-03-02 15:00:00,18.0,13.0,42.0,6.0,5.0,watch meet press treat climate change big joke...,-0.952381,3.492063,4.444444,7.936508,26.984127,1.428571,576.000000,0,0.0
2,6,2015-03-02 16:00:00,17.0,13.0,41.0,6.0,5.0,no_titlemary bowerman usa today network visito...,0.000000,1.814059,1.814059,3.628118,25.396825,0.000000,405.000000,0,0.0
3,8,2015-03-02 18:00:00,18.0,14.0,39.0,6.0,4.0,russian energy deal come contentious timemr. f...,-1.147541,1.803279,2.950820,4.754098,19.508197,0.409836,1119.000000,0,0.0
4,10,2015-03-02 20:00:00,17.0,13.0,35.0,5.0,4.0,climate change cause syrian civil warclimate c...,-5.368873,0.909366,6.278239,7.187605,24.159231,0.123712,616.333333,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29384,48471,2020-10-07 16:00:00,14.5,17.0,54.0,11.0,8.5,hurricane delta landfall gulf coast category 3...,-1.933350,1.456169,3.389519,4.845688,22.040880,0.675240,656.857143,0,0.0
29385,48472,2020-10-07 17:00:00,13.5,17.5,54.0,10.5,8.5,150 million join extreme poor world bank© prov...,-1.897305,2.429582,4.326887,6.756470,22.310394,0.693847,923.571429,0,0.0
29386,48473,2020-10-07 18:00:00,13.5,17.5,50.0,10.5,8.0,china 2060 carbon neutral goal bill hit $ 5 tr...,-1.070552,1.935600,3.006152,4.941752,22.293919,0.268835,561.750000,0,0.0
29387,48474,2020-10-07 19:00:00,13.5,17.5,46.0,10.5,7.0,dare city 2020 kejriwal speak ev policy odd sc...,-4.013967,1.189192,5.203159,6.392350,21.418546,0.554424,578.000000,0,0.0


In [27]:
#checking target column
df['shifted_12h']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
29384    0.0
29385    0.0
29386    0.0
29387    0.0
29388    0.0
Name: shifted_12h, Length: 29389, dtype: float64

In [28]:
print('setting x to title_text and y to shifte_12h')
X = df['title_text']
y = df['shifted_12h'].astype(int)
display(f'x_shape: {X.shape}')
display(f'y_shape: {y.shape}')

print('creating test train split')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y)

print(f'X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}')

setting x to title_text and y to shifte_12h


'x_shape: (29389,)'

'y_shape: (29389,)'

creating test train split
X_train: (20572,), y_train: (20572,), X_test: (8817,), y_test: (8817,)


In [14]:
#download and set word vector pretrained model
embedding = "https://tfhub.dev/google/nnlm-en-dim128/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

print('Google news model downlownded')

Google news model downlownded


In [29]:
#define feed forward neural network with one embed layer and 4 dense layers
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))

#compile model
model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
print('model defined and compiled')

model defined and compiled


In [30]:
#train model
print('training model')
history = model.fit(X_train, y_train, epochs=3, verbose=1, validation_data=(X_test, y_test))
print('model training complete')

training model
Epoch 1/3
643/643 [==============================] - 85s 132ms/step - loss: 0.5011 - accuracy: 0.7212 - val_loss: 0.3419 - val_accuracy: 0.8742
Epoch 2/3
643/643 [==============================] - 85s 132ms/step - loss: 0.3696 - accuracy: 0.8449 - val_loss: 0.3075 - val_accuracy: 0.8840
Epoch 3/3
643/643 [==============================] - 85s 132ms/step - loss: 0.2999 - accuracy: 0.8822 - val_loss: 0.3248 - val_accuracy: 0.8689
model training complete


In [31]:
#save model
model.save('article_gtrend_12_9_20/')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: article_gtrend_12_9_20/assets


INFO:tensorflow:Assets written to: article_gtrend_12_9_20/assets


In [36]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense_20 (Dense)             (None, 16)                2064      
_________________________________________________________________
dense_21 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_22 (Dense)             (None, 4)                 36        
_________________________________________________________________
dropout_5 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 5         
Total params: 124,644,929
Trainable params: 124,644,929
Non-trainable params: 0
________________________________________

In [47]:
#generate predictions on test set
test_pred = (model.predict(X_test) > 0.5).astype("int32")

In [48]:
from sklearn.metrics import accuracy_score

In [49]:
#calculate test accuracy
accuracy_score(y_test, test_pred)

0.8688896450039696

In [50]:
from sklearn.metrics import classification_report

In [53]:
#generate classification report on test data
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.83      0.91      0.87      4102
           1       0.91      0.83      0.87      4715

    accuracy                           0.87      8817
   macro avg       0.87      0.87      0.87      8817
weighted avg       0.87      0.87      0.87      8817



In [55]:
#checking saved model
reloaded = tf.keras.models.load_model('article_gtrend_12_9_20')

In [56]:
reloaded.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense_20 (Dense)             (None, 16)                2064      
_________________________________________________________________
dense_21 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_22 (Dense)             (None, 4)                 36        
_________________________________________________________________
dropout_5 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 5         
Total params: 124,644,929
Trainable params: 124,644,929
Non-trainable params: 0
________________________________________

In [57]:
reloaded.evaluate(X_test, y_test)

276/276 [==============================] - 5s 19ms/step - loss: 0.3248 - accuracy: 0.8689


[0.3248368203639984, 0.8688896298408508]